In [1]:
import setup
setup.init_django()

In [2]:
from products.models import Product,Embedding
from sentence_transformers import SentenceTransformer

In [3]:
from openai import OpenAI
from decouple import config
EMBEDDING_MODEL=config("EMBEDDING_MODEL", default="text-embedding-3-small")
RECREATE_DATA=True

In [4]:
client=OpenAI(
    api_key=config("OPENAI_API_KEY")
)

In [5]:
def get_embedding(text,model=EMBEDDING_MODEL):
    text=text.replace("\n","").strip()
    return client.embeddings.create(input=[text],model=model).data[0].embedding

In [6]:
query="The dog jumped over the green cow"
# query="The dog jumped over the cat"
query_embedding=get_embedding(query)
query_embedding

[0.03823733329772949,
 -0.017965277656912804,
 0.027054481208324432,
 -0.005024134647101164,
 -0.046060316264629364,
 -0.003958504181355238,
 0.010969726368784904,
 0.022516148164868355,
 0.04137153923511505,
 -0.01599699631333351,
 0.01595938578248024,
 0.010712721385061741,
 -0.042600151151418686,
 0.02449696697294712,
 -0.0467623770236969,
 -0.009233376011252403,
 -0.011421052739024162,
 -0.032846495509147644,
 -0.06188179552555084,
 0.01419169269502163,
 -0.04568421095609665,
 0.04307654872536659,
 -0.030715234577655792,
 0.020974118262529373,
 0.008731902576982975,
 -0.029536772519350052,
 0.030489571392536163,
 0.014204229228198528,
 0.018893003463745117,
 -0.004403561819344759,
 -0.022002138197422028,
 -0.024020567536354065,
 0.0013077481416985393,
 -0.07176081836223602,
 -0.00920830201357603,
 -0.026477785781025887,
 -0.024446818977594376,
 -0.01661130040884018,
 -0.0049050347879529,
 0.04731399938464165,
 0.011283147148787975,
 -0.016473395749926567,
 0.01710023730993271,
 -0.

In [9]:
from pgvector.django import CosineDistance
from django.db.models import F
qs=Embedding.objects.annotate(distance=CosineDistance('embedding',query_embedding),
                             similarity=1-F("distance")).order_by("distance")[:2]
for obj in qs:
    print(obj.object_id, obj.content_object.id,obj.content_object.title, obj.distance, obj.similarity*100)

8 8 Blog Post 4 0.6165880446609271 38.34119553390729
1 1 random product 0.6165880446609271 38.34119553390729
